# Exploring the mechanism of `Exchange`

In [2]:
from datetime import datetime, timedelta

import os
os.chdir("..")  # path to the base directory of rl4mm
from rl4mm.orderbook.create_order import create_order, OrderDict
from rl4mm.orderbook.models import Order, Orderbook, LimitOrder, MarketOrder, Cancellation, Deletion
from rl4mm.orderbook.Exchange import *
from rl4mm.orderbook.helpers import *

# Jedi not working
%config Completer.use_jedi = False

In [21]:
import timeit
import time

In [22]:
ticker="FAKE"
start = datetime(2019,1,2,10,35,45)
delta = timedelta(microseconds = 100000)

## To manage an Orderbook, create an ```Exchange```

each exchange automatically initialises an ```Orderbook``` to keep track of.

In [129]:
n_runs_timing=100000
exchange = Exchange(ticker)
orderbook = exchange.central_orderbook
exchanges=[Exchange(ticker) for i in range(n_runs_timing)]

## Make some tools to create orders of different types

In [70]:
def get_limit_order(timestamp:datetime, price:float, volume:int, direction:str, internal_id: int):
    return LimitOrder(timestamp = timestamp, price = int(100*price), volume=volume, direction=direction, ticker = "MSFT,=", internal_id=internal_id, external_id=None, is_external= True)
def get_market_order(timestamp:datetime, volume:int, direction:str, internal_id: int=None):
    return MarketOrder(timestamp = timestamp, volume=volume, direction=direction, ticker = "MSFT,=", internal_id=internal_id, external_id=None, is_external= True)
def get_cancellation(timestamp:datetime, price: float, volume:int, direction:str, internal_id:int):
    return Cancellation(timestamp = timestamp, price = int(100*price), volume=volume, direction=direction, ticker = "MSFT,=",  internal_id = internal_id, external_id=None, is_external= True)
def get_deletion(timestamp:datetime, price: float, direction:str, internal_id):
    return Deletion(timestamp = timestamp, price = int(100*price), volume = None, direction=direction, ticker = "MSFT", internal_id = internal_id, external_id=None, is_external= True)

## Submitting limit orders

In [130]:
limit_buy_price_vol_tuples = ((99.8,10), (99.85, 50), (99.87, 100), (99.9, 80), (99.95, 100), (99.98, 200))
limit_sell_price_vol_tuples = ((100.1,20), (100.05, 30), (100.04, 60), (100.02, 80), (100.01, 150))
internal_id = 1
ts = start
limit_orders = list()
for price, vol in limit_buy_price_vol_tuples:
    limit_orders.append(get_limit_order(ts, price, vol, "buy", internal_id))
    internal_id +=1
    ts +=delta
for price, vol in limit_sell_price_vol_tuples:
    limit_orders.append(get_limit_order(ts, price, vol, "sell", internal_id))
    internal_id +=1
    ts +=delta       

In [79]:
n_runs=10000
timeit.timeit("exchange.process_order(limit_orders[0])",number=n_runs,globals=globals())/n_runs

2.87684581708163e-06

In [131]:
for order in limit_orders:
        exchange.process_order(order)



s=time.time()
for i,e in enumerate(exchanges):
    for order in limit_orders:
        e.process_order(order)
en=time.time()
print((en-s)/len(exchanges)/len(limit_orders))
print(len(limit_orders))
visualise_orderbook(exchange.central_orderbook, price_scaling = 1/100)

4.910327304493298e-06
11


## Market orders

In [132]:
market_order = get_market_order(direction="sell", volume=150, timestamp=ts)
ts += delta
exchange.process_order(market_order)
visualise_orderbook(exchange.central_orderbook)

In [133]:
market_order = get_market_order(direction="sell", volume=150, timestamp=ts)
s=time.time()
for i,e in enumerate(exchanges):
    e.process_order(market_order)
en=time.time()
print((en-s)/len(exchanges))

1.6036412715911865e-05


### Trying to place a market order that takes all the volume from the book, and more

Note that perhaps this is not the desired behaviour and we should let the market order take all the liquidity (as we currently do) but then just ignore the rest. To discuss.

In [88]:
market_order = get_market_order(direction="sell", volume=700, timestamp=ts)
ts += delta
try:
    exchange.process_order(market_order)
except EmptyOrderbookError:
    print("Volume too large!")    

Volume too large!


## Cancellations and Deletions

In [150]:
# reset orderbook
exchange.central_orderbook = exchange.get_empty_orderbook()
for order in limit_orders:
    exchange.process_order(order)


s=time.time()
for i,e in enumerate(exchanges):
    e.central_orderbook = e.get_empty_orderbook()
    for order in limit_orders:
        e.process_order(order)
en=time.time()
print((en-s)/len(exchanges)/len(limit_orders))
print(len(limit_orders))
visualise_orderbook(exchanges[0].central_orderbook, price_scaling = 1/100)

1.3222045464949176e-05
11


In [151]:
exchange.central_orderbook.buy

SortedDict({9980: deque([LimitOrder(timestamp=datetime.datetime(2019, 1, 2, 10, 35, 45), direction='buy', ticker='MSFT,=', internal_id=23, external_id=None, is_external=True, price=9980, volume=10)]), 9985: deque([LimitOrder(timestamp=datetime.datetime(2019, 1, 2, 10, 35, 45, 100000), direction='buy', ticker='MSFT,=', internal_id=24, external_id=None, is_external=True, price=9985, volume=50)]), 9987: deque([LimitOrder(timestamp=datetime.datetime(2019, 1, 2, 10, 35, 45, 200000), direction='buy', ticker='MSFT,=', internal_id=25, external_id=None, is_external=True, price=9987, volume=100)]), 9990: deque([LimitOrder(timestamp=datetime.datetime(2019, 1, 2, 10, 35, 45, 300000), direction='buy', ticker='MSFT,=', internal_id=26, external_id=None, is_external=True, price=9990, volume=80)]), 9995: deque([LimitOrder(timestamp=datetime.datetime(2019, 1, 2, 10, 35, 45, 400000), direction='buy', ticker='MSFT,=', internal_id=27, external_id=None, is_external=True, price=9995, volume=100)]), 9998: deq

In [152]:
# We cancel half of the buy order at level 99.9
order = exchange.central_orderbook.buy[int(99.9*100)][0]
cancellation = get_cancellation(timestamp=ts, price = (order.price)/100, direction="buy", internal_id=order.internal_id, volume=order.volume/2)
ts += delta

In [153]:
# We cancel half of the buy order at level 99.9
order2 = exchanges[0].central_orderbook.buy[int(99.9*100)][0]
cancellation2 = get_cancellation(timestamp=ts, price = (order2.price)/100, direction="buy", internal_id=order2.internal_id, volume=order2.volume/2)
ts += delta

In [154]:
print(order2)

LimitOrder(timestamp=datetime.datetime(2019, 1, 2, 10, 35, 45, 300000), direction='buy', ticker='MSFT,=', internal_id=26, external_id=None, is_external=True, price=9990, volume=80)


In [155]:
order = exchange.central_orderbook.buy[int(99.9*100)][0]

In [156]:
cancellation2

Cancellation(timestamp=datetime.datetime(2019, 1, 2, 10, 35, 46, 600000), direction='buy', ticker='MSFT,=', internal_id=26, external_id=None, is_external=True, price=9990, volume=40.0)

In [157]:
exchange.process_order(cancellation)

s=time.time()
for i,e in enumerate(exchanges):
    e.process_order(cancellation2)
en=time.time()
print((en-s)/len(exchanges))

1.6117618083953856e-05


In [158]:
visualise_orderbook(exchanges[0].central_orderbook)

In [159]:
# Delete best order
order = exchanges[0].central_orderbook.buy[9998][0]
deletion = get_deletion(timestamp=ts, price = (order.price)/100, direction="buy", internal_id=order.internal_id)
ts += delta

In [160]:
deletion

Deletion(timestamp=datetime.datetime(2019, 1, 2, 10, 35, 46, 700000), direction='buy', ticker='MSFT', internal_id=28, external_id=None, is_external=True, price=9998, volume=None)

In [161]:
s=time.time()
for i,e in enumerate(exchanges):
    e.process_order(deletion)
en=time.time()
print((en-s)/len(exchanges))

1.813880681991577e-05


In [162]:
visualise_orderbook(exchanges[0].central_orderbook)

## Crossing the spread with a limit order

In [166]:
# reset orderbook
exchange.central_orderbook = exchange.get_empty_orderbook()
for order in limit_orders:
    exchange.process_order(order)


s=time.time()
for i,e in enumerate(exchanges):
    e.central_orderbook = e.get_empty_orderbook()
    for order in limit_orders:
        e.process_order(order)
en=time.time()
print((en-s)/len(exchanges)/len(limit_orders))
print(len(limit_orders))
visualise_orderbook(exchanges[0].central_orderbook, price_scaling = 1/100)

1.2626417116685348e-05
11


In [167]:
limit_cross = get_limit_order(start, 100.03, 300, "buy", internal_id)
internal_id +=1


s=time.time()
for i,e in enumerate(exchanges):
    e.process_order(limit_cross)
en=time.time()
print((en-s)/len(exchanges))

visualise_orderbook(exchanges[0].central_orderbook)

3.7944259643554684e-05
